In [ ]:
#run if requirements needed !pip install -r requirements.txt

In [19]:
import os
import concurrent.futures
import openai
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric, remove_stopwords
df = pd.read_csv("NEWS_YAHOO_stock_prediction.csv")
import time
from tqdm import tqdm


In [20]:
df.drop(columns="Unnamed: 0",inplace=True)
df.set_index("Date")


,ticker,category,title,content,Open,High,Low,Close,Adj Close,Volume,label
Date,,,,,,,,,,,
2020-01-27,AAPL,opinion,Apple Set To Beat Q1 Earnings Estimates Tech ...,Technology giant Apple NASDAQ AAPL is set ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Tech Daily Intel Results Netflix Surge Appl...,The top stories in this digest are Intel s N...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,7 Monster Stock Market Predictions For The Wee...,S P 500 SPY \nThis week will be packed with e...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Apple Earnings Preview 5G Launch Expanding S...,Reports Q1 2020 results on Tuesday Jan 28 ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Buy Surging Apple Microsoft Stock Before Qua...,On today s episode of Full Court Finance here ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
...,...,...,...,...,...,...,...,...,...,...,...
2012-07-31,AAPL,opinion,What s Driving China s Real Estate Rally Par...,In the preceding posts I examined the first t...,21.543928,21.846430,21.525715,21.812857,18.624512,462327600,1
2012-07-31,AAPL,opinion,50 Stocks With The Best Dividend Growth In Jul...,Here is a current sheet of companies with fast...,21.543928,21.846430,21.525715,21.812857,18.624512,462327600,1
2012-07-23,AAPL,opinion,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,21.228571,21.639286,20.989643,21.565357,18.413187,487975600,1


In [21]:

# Load dataset
df = df.dropna(subset=["content"])

# Preprocess text (custom pipeline)
def preprocess_text(text):
    filters = [lambda x: x.lower(), strip_punctuation, strip_numeric, remove_stopwords]
    return preprocess_string(text, filters)

df["cleaned_text"] = df["content"].apply(preprocess_text)

# Create dictionary and corpus
dictionary = corpora.Dictionary(df["cleaned_text"])
corpus = [dictionary.doc2bow(text) for text in df["cleaned_text"]]

# Number of topics
num_topics = 5

# Train LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

# Extract topic affinities
topic_affinities = []
for doc in corpus:
    topics = lda_model.get_document_topics(doc, minimum_probability=0)
    topic_affinities.append([prob for _, prob in topics])

# Convert to DataFrame
df_topics = pd.DataFrame(topic_affinities, columns=[f"Topic_{i}" for i in range(num_topics)])
df_final = pd.concat([df, df_topics], axis=1)

# Save and display
df_final.to_csv("news_with_topics_gensim.csv", index=False)




In [27]:
import pandas as pd
import concurrent.futures
from tqdm import tqdm
from openai import OpenAI

# Load API key from file
with open("api.txt", "r") as file:
    api_key = file.read().strip()

# Initialize OpenAI client with OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    default_headers={  # Set headers here, as OpenRouter recommends
        "HTTP-Referer": "https://your-site.com",  # Optional: Replace with your site URL
        "X-Title": "Your Site Name",  # Optional: Replace with your site name
    }
)

In [32]:
def extract_scores(reply):
    """
    Use regex to extract the first float following 'Sentiment:' and 'Importance:'.
    """
    sentiment_pattern = r"Sentiment:\s*([-+]?\d*\.?\d+)"
    importance_pattern = r"Importance:\s*([-+]?\d*\.?\d+)"

    sentiment_match = re.search(sentiment_pattern, reply)
    importance_match = re.search(importance_pattern, reply)

    if sentiment_match and importance_match:
        sentiment_score = float(sentiment_match.group(1))
        importance_score = float(importance_match.group(1))
        return sentiment_score, importance_score
    else:
        raise ValueError("Could not extract scores from reply: " + reply)

def get_scores(row):
    """
    Function to get sentiment and importance scores for a row of content and title.
    """
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the following article:
        Title: {row['title']}
        Content: {row['content']}
        
        Provide two scores:
        1. Sentiment score (scale -1 to 1): How positive or negative is the sentiment of the article?
        2. Importance score (scale 0 to 1): How important is this article's content for its context?
        
        Format your response as:
        Sentiment: <sentiment_score>
        Importance: <importance_score>
        """

        # Call the OpenRouter API using Qwen
        response = client.chat.completions.create(
            model="qwen/qwen-turbo",  # Corrected model name
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.7
        )

        # Check if the response contains valid choices
        if not response or not hasattr(response, "choices") or len(response.choices) == 0:
            raise ValueError("No valid choices in API response.")

        reply = response.choices[0].message.content.strip()
        # Use regex to extract the scores
        sentiment_score, importance_score = extract_scores(reply)
        return sentiment_score, importance_score

    except Exception as e:
        print(f"Error processing row: {e}")
        return None, None

def process_dataset_in_parallel(df, num_workers=1):
    """
    Processes a dataset in parallel using the scoring function.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(executor.map(get_scores, [row for _, row in df.iterrows()]),
                            total=len(df), desc="Processing"))
    
    # Unpack the results into two new columns
    df['sentiment_score'], df['importance_score'] = zip(*results)
    return df

# Load your dataset
# For example: df = pd.read_csv("your_dataset.csv")
# Here, we assume 'df' is already defined
d = df.sample(5)  # Sample 5 rows for testing

# Process the dataset
d = process_dataset_in_parallel(d)

# Save or print the updated dataset
print("Processing complete. Scores added to dataset.")

Processing: 100%|██████████| 5/5 [00:17<00:00,  3.57s/it]

Processing complete. Scores added to dataset.


In [33]:
d

,ticker,Date,category,title,content,Open,High,Low,Close,Adj Close,Volume,label,cleaned_text,sentiment_score,importance_score
7414,AAPL,2018-01-29,opinion,USD Index Trades Heavily Ahead Of This Week s ...,We can look back on the week that was and feel...,42.540001,42.540001,41.767502,41.990002,39.994514,202561600,0,"[look, week, feel, small, way, cross, currents...",0.0,0.8
4516,AAPL,2018-11-29,opinion,Here s What To Expect From VMware s VMW Q3 E...,Shares of VMware Inc NYSE VMW jumped nea...,45.665001,45.700001,44.424999,44.887501,43.396198,167080000,0,"[shares, vmware, nyse, vmw, jumped, nearly, re...",0.4,0.7
11186,AAPL,2017-05-04,opinion,Is A Surprise In Store For Model N MODN In Q...,Model N Inc NYSE MODN is scheduled to rep...,36.055000,36.365002,35.952499,36.005001,33.897518,110871600,0,"[model, n, nyse, modn, scheduled, report, seco...",0.2,0.7
3549,AAPL,2019-01-23,opinion,F5 Networks FFIV Q1 Earnings Top Estimates ...,F5 Networks Inc NASDAQ FFIV delivered fir...,38.537498,38.785000,37.924999,38.480000,37.201569,92522400,0,"[f, networks, nasdaq, ffiv, delivered, quarter...",0.6,0.7
14062,AAPL,2015-06-11,news,Japan messaging app Line launches music stream...,By Joshua Hunt TOKYO Reuters Messaging app...,30.277500,30.452499,30.155001,30.264999,27.753017,132169200,0,"[joshua, hunt, tokyo, reuters, messaging, app,...",0.4,0.7
